In [ ]:
from lib.NoSQLBench import RunCommand
from Runner import Runner

import time
import logging
import sys
import enoslib as en
import enoslib.config as en_config

In [ ]:
en_config.set_config(g5k_cache=False, ansible_stdout="noop")

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
runner = Runner()

In [ ]:
runner.get_resources(site="nancy",
                     cluster="gros",
                     walltime="1:00:00",
                     job_name="cassandra",
                     node_count=1,
                     seed_count=1,
                     client_count=1)

In [ ]:
runner.deploy_cassandra(name="cassandra",
                        docker_image="adugois1/apache-cassandra-base:latest",
                        conf_template="templates/cassandra/conf/cassandra-base.yaml")

In [ ]:
runner.deploy_nb(name="nosqlbench",
                 docker_image="nosqlbench/nosqlbench:nb5preview")

In [ ]:
runner.run_nb(
    RunCommand.from_options(driver="cqld4",
                            workload=f"{runner.nb.container_conf_path}/workloads/baseline",
                            tags="block:schema",
                            threads=1,
                            rf=1,
                            host=runner.cassandra.hosts[0].address,
                            localdc="datacenter1")
)

In [ ]:
runner.run_nb(
    RunCommand.from_options(driver="cqld4",
                            workload=f"{runner.nb.container_conf_path}/workloads/baseline",
                            tags="block:rampup",
                            threads="auto",
                            cycles=100000000,
                            cyclerate=50000,
                            keycount=100000000,
                            host=runner.cassandra.hosts[0].address,
                            localdc="datacenter1")
)

In [ ]:
logging.info(runner.cassandra.nodetool("tablestats baselines.keyvalue"))
logging.info(runner.cassandra.du("/var/lib/cassandra/data/baselines"))

In [ ]:
with en.Dstat(nodes=runner.nb.hosts, options="-aT", backup_dir="./results/base-80/dstat") as dstat:
    time.sleep(5)

    runner.run_nb(
        RunCommand.from_options(driver="cqld4",
                                workload=f"{runner.nb.container_conf_path}/workloads/baseline",
                                tags="block:main-read",
                                threads="auto",
                                cycles=10000000,
                                cyclerate=11129,
                                keycount=100000000,
                                host=runner.cassandra.hosts[0].address,
                                localdc="datacenter1")
        .logs_dir(runner.nb.container_data_path)
        .log_histograms(f"{runner.nb.container_data_path}/histograms.csv")
        .log_histostats(f"{runner.nb.container_data_path}/histostats.csv")
        .report_summary_to(f"{runner.nb.container_data_path}/summary.txt")
        .report_csv_to(f"{runner.nb.container_data_path}/csv")
        .report_interval(30)
    )

    time.sleep(5)

In [ ]:
runner.nb.sync_results(dest="./results/base-80")